In [1]:
import os
import mypackages.myrasters as mr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
wcorr_dir = os.path.join('..', 'output/corrections_calculated')
clay_dir = os.path.join('..', 'output/soilgrids_prepared')
out_dir = wcorr_dir

In [3]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun',
          'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

The clay layer has been resampled to the resolution of the soil water content rasters in ArcGIS.

In [4]:
filename = 'CLYPPT_ts_resampled.tif'
clay_raster = mr.MyRaster(os.path.join(clay_dir, filename))

In [5]:
clay = clay_raster.get_array()

In [6]:
clay = clay / 10

In [7]:
%%time

ts_monthly =[]

for i in range(0, 12):
    filename = 'ts_' + str(i+1) + '_wcorr_1e6_' + months[i] + '.tif'
    raster = mr.MyRaster(os.path.join(wcorr_dir, filename))
    wcorr = raster.get_array()
    ts = wcorr * clay
    ts[wcorr == raster.NoDataValue] = raster.NoDataValue
    ts[clay == clay_raster.NoDataValue / 10] = raster.NoDataValue
    ts = ts.astype('uint16')
    
    ts_monthly.append(ts)
    
    out_filename = 'ts_' + str(i+1) + '_claycorr_1e5_' + months[i] + '.tif'
    mr.array2geotiff_rastercopy(ts, os.path.join(out_dir, out_filename), raster.raster)

ts_monthly = np.array(ts_monthly)

CPU times: user 6min 5s, sys: 47.6 s, total: 6min 52s
Wall time: 6min 53s


In [8]:
%%time

array = ts_monthly.mean(axis=0)
array = array.round(0)
array = array.astype('uint16')
out_filename = 'ts_claycorr_1e5_yearly_mean'

mr.array2geotiff_rastercopy(array, os.path.join(out_dir, out_filename), raster.raster)
                 

CPU times: user 35.6 s, sys: 3.46 s, total: 39.1 s
Wall time: 39.1 s


In [9]:
%%time

array = ts_monthly.std(axis=0)
array = array.round(0)
array = array.astype('uint16')
array[ts_monthly[0] == raster.NoDataValue] = raster.NoDataValue

out_filename = 'ts_claycorr_1e5_yearly_std'

mr.array2geotiff_rastercopy(array, os.path.join(out_dir, out_filename), raster.raster)


CPU times: user 1min 45s, sys: 1min 39s, total: 3min 24s
Wall time: 3min 24s
